In [18]:
import pandas as pd
import re
import nltk
import spacy

nltk.download("stopwords")
nlp = spacy.load("en_core_web_sm")
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beste\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\beste\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\beste\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
path = "Scripts TBBT.csv"
df = pd.read_csv(path , sep = "," , skipinitialspace=True, engine="python")

df = df[df['person_scene'].str.contains('Amy')]
df

,episode_name,dialogue,person_scene
15116,Series 03 Episode 23 – The Lunar Excitation,Excuse me. I’m Amy Farrah Fowler. You’re Sheld...,Amy
15118,Series 03 Episode 23 – The Lunar Excitation,"If that was slang, I’m unfamiliar with it. If...",Amy
15120,Series 03 Episode 23 – The Lunar Excitation,"I don’t object to the concept of a deity, but...",Amy
15122,Series 03 Episode 23 – The Lunar Excitation,"Noted. Now, before this goes any further, you...",Amy
15124,Series 03 Episode 23 – The Lunar Excitation,"Tepid water, please.",Amy
...,...,...,...
54344,Series 10 Episode 24 – The Long Distance Disso...,I was missing you again.,Amy
54346,Series 10 Episode 24 – The Long Distance Disso...,She’s what?,Amy
54348,Series 10 Episode 24 – The Long Distance Disso...,Hi.,Amy
54350,Series 10 Episode 24 – The Long Distance Disso...,Uh-huh. Hang on. (answering phone) Hello. A l...,Amy


In [4]:
data = df['dialogue']
data

15116    Excuse me. I’m Amy Farrah Fowler. You’re Sheld...
15118     If that was slang, I’m unfamiliar with it. If...
15120     I don’t object to the concept of a deity, but...
15122     Noted. Now, before this goes any further, you...
15124                                 Tepid water, please.
                               ...                        
54344                             I was missing you again.
54346                                          She’s what?
54348                                                  Hi.
54350     Uh-huh. Hang on. (answering phone) Hello. A l...
54352                                  It’s not important.
Name: dialogue, Length: 3420, dtype: object

In [5]:
def clean_text(text):
    # clean punctuation
    text = re.sub(r"[’!”#$%&()*+,-.:;?@[\]^_`{|}~]", "", str(text))
    # lower
    return text.lower()

In [13]:
def process_words(text):
    # tokenization
    sentences = nltk.tokenize.sent_tokenize(text)
    words = nltk.tokenize.word_tokenize(text)
    # remove stopwords
    stop_words = set(nltk.corpus.stopwords.words('english'))
    filter_words = [word for word in words if word not in stop_words]
    # stemming
    porter = nltk.stem.PorterStemmer()
    stemmed_words = [porter.stem(word) for word in filter_words]
    return {
        "sentences" : sentences,
        "filtered_words" : filter_words,
        "stemmed_words" : stemmed_words
    }

In [14]:
def pos_ner(text):
    doc = nlp(text)
    pos_tag = [(token.text, token.pos_) for token in doc]
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return {
        "pos_tags": pos_tag,
        "entities" : entities
    }

In [19]:
processed_data = []
for text in data:
    cleaned = clean_text(text)
    processed = process_words(cleaned)
    ner_pos = pos_ner(" ".join(processed["filtered_words"]))
    processed_data.append({
        "org_text": text,
        "cleaned_text": cleaned,
        "filtered_words": processed["filtered_words"],
        "stemmed_words": processed["stemmed_words"],
        "pos_tags": ner_pos["pos_tags"],
        'entities': ner_pos['entities']
    }
    )

In [20]:
processed_data

[{'org_text': 'Excuse me. I’m Amy Farrah Fowler. You’re Sheldon Cooper.',
  'cleaned_text': 'excuse me im amy farrah fowler youre sheldon cooper',
  'filtered_words': ['excuse',
   'im',
   'amy',
   'farrah',
   'fowler',
   'youre',
   'sheldon',
   'cooper'],
  'stemmed_words': ['excus',
   'im',
   'ami',
   'farrah',
   'fowler',
   'your',
   'sheldon',
   'cooper'],
  'pos_tags': [('excuse', 'NOUN'),
   ('i', 'PRON'),
   ('m', 'VERB'),
   ('amy', 'ADV'),
   ('farrah', 'PROPN'),
   ('fowler', 'PROPN'),
   ('you', 'PRON'),
   ('re', 'VERB'),
   ('sheldon', 'PROPN'),
   ('cooper', 'NOUN')],
  'entities': [('farrah fowler', 'PERSON')]},
 {'org_text': ' If that was slang, I’m unfamiliar with it. If it was literal, I share your aversion to soiled hosiery. In any case, I’m here because my mother and I have agreed that I will date at least once a year.',
  'cleaned_text': ' if that was slang im unfamiliar with it if it was literal i share your aversion to soiled hosiery in any case im h

In [21]:
for i, data in enumerate(processed_data):
    if "pos_tags" not in data:
        print(f"Missing 'pos_tags' in entry {i}: {data}")

In [ ]:
df['filtered_words'] = [data['filtered_words'] for data in processed_data]
df['stemmed_words'] = [data['stemmed_words'] for data in processed_data]
df['pos_tags'] = [data["pos_tags"] for data in processed_data]
df['entities'] = [data['entities'] for data in processed_data]


,episode_name,dialogue,person_scene,filtered_words,stemmed_words,pos_tags,entities
15116,Series 03 Episode 23 – The Lunar Excitation,Excuse me. I’m Amy Farrah Fowler. You’re Sheld...,Amy,"[excuse, im, amy, farrah, fowler, youre, sheld...","[excus, im, ami, farrah, fowler, your, sheldon...","[(excuse, NOUN), (i, PRON), (m, VERB), (amy, A...","[(farrah fowler, PERSON)]"
15118,Series 03 Episode 23 – The Lunar Excitation,"If that was slang, I’m unfamiliar with it. If...",Amy,"[slang, im, unfamiliar, literal, share, aversi...","[slang, im, unfamiliar, liter, share, avers, s...","[(slang, PROPN), (i, PRON), (m, VERB), (unfami...","[(slang, PERSON), (date least year, DATE)]"
15120,Series 03 Episode 23 – The Lunar Excitation,"I don’t object to the concept of a deity, but...",Amy,"[dont, object, concept, deity, im, baffled, no...","[dont, object, concept, deiti, im, baffl, noti...","[(do, AUX), (nt, PART), (object, VERB), (conce...",[]
15122,Series 03 Episode 23 – The Lunar Excitation,"Noted. Now, before this goes any further, you...",Amy,"[noted, goes, know, forms, physical, contact, ...","[note, goe, know, form, physic, contact, inclu...","[(noted, VERB), (goes, VERB), (know, VERB), (f...",[]
15124,Series 03 Episode 23 – The Lunar Excitation,"Tepid water, please.",Amy,"[tepid, water, please]","[tepid, water, pleas]","[(tepid, NOUN), (water, NOUN), (please, INTJ)]",[]
...,...,...,...,...,...,...,...
54344,Series 10 Episode 24 – The Long Distance Disso...,I was missing you again.,Amy,[missing],[miss],"[(missing, VERB)]",[]
54346,Series 10 Episode 24 – The Long Distance Disso...,She’s what?,Amy,[shes],[she],"[(she, PRON), (s, VERB)]",[]
54348,Series 10 Episode 24 – The Long Distance Disso...,Hi.,Amy,[hi],[hi],"[(hi, INTJ)]",[]
54350,Series 10 Episode 24 – The Long Distance Disso...,Uh-huh. Hang on. (answering phone) Hello. A l...,Amy,"[uhhuh, hang, answering, phone, hello, little,...","[uhhuh, hang, answer, phone, hello, littl, lat...","[(uhhuh, PROPN), (hang, NOUN), (answering, VER...",[]


In [54]:
df.to_csv('processed_data.csv', index=False)

In [ ]:
# Question 1 

df['season'] = df['episode_name'].str.extract(r'Series (\d+)').astype(int)
df['episode'] = df['episode_name'].str.extract(r'Episode (\d+)').astype(int)


,episode_name,dialogue,person_scene,filtered_words,stemmed_words,pos_tags,entities,season,episode
15116,Series 03 Episode 23 – The Lunar Excitation,Excuse me. I’m Amy Farrah Fowler. You’re Sheld...,Amy,"[excuse, im, amy, farrah, fowler, youre, sheld...","[excus, im, ami, farrah, fowler, your, sheldon...","[(excuse, NOUN), (i, PRON), (m, VERB), (amy, A...","[(farrah fowler, PERSON)]",03,23
15118,Series 03 Episode 23 – The Lunar Excitation,"If that was slang, I’m unfamiliar with it. If...",Amy,"[slang, im, unfamiliar, literal, share, aversi...","[slang, im, unfamiliar, liter, share, avers, s...","[(slang, PROPN), (i, PRON), (m, VERB), (unfami...","[(slang, PERSON), (date least year, DATE)]",03,23
15120,Series 03 Episode 23 – The Lunar Excitation,"I don’t object to the concept of a deity, but...",Amy,"[dont, object, concept, deity, im, baffled, no...","[dont, object, concept, deiti, im, baffl, noti...","[(do, AUX), (nt, PART), (object, VERB), (conce...",[],03,23
15122,Series 03 Episode 23 – The Lunar Excitation,"Noted. Now, before this goes any further, you...",Amy,"[noted, goes, know, forms, physical, contact, ...","[note, goe, know, form, physic, contact, inclu...","[(noted, VERB), (goes, VERB), (know, VERB), (f...",[],03,23
15124,Series 03 Episode 23 – The Lunar Excitation,"Tepid water, please.",Amy,"[tepid, water, please]","[tepid, water, pleas]","[(tepid, NOUN), (water, NOUN), (please, INTJ)]",[],03,23
...,...,...,...,...,...,...,...,...,...
54344,Series 10 Episode 24 – The Long Distance Disso...,I was missing you again.,Amy,[missing],[miss],"[(missing, VERB)]",[],10,24
54346,Series 10 Episode 24 – The Long Distance Disso...,She’s what?,Amy,[shes],[she],"[(she, PRON), (s, VERB)]",[],10,24
54348,Series 10 Episode 24 – The Long Distance Disso...,Hi.,Amy,[hi],[hi],"[(hi, INTJ)]",[],10,24
54350,Series 10 Episode 24 – The Long Distance Disso...,Uh-huh. Hang on. (answering phone) Hello. A l...,Amy,"[uhhuh, hang, answering, phone, hello, little,...","[uhhuh, hang, answer, phone, hello, littl, lat...","[(uhhuh, PROPN), (hang, NOUN), (answering, VER...",[],10,24


In [36]:

def calculate_counts(dialogue):
    sentences = nltk.sent_tokenize(dialogue)
    words = nltk.word_tokenize(dialogue)
    return len(sentences), len(words)

# Apply to calculate sentence and word counts
df[['sentence_count', 'word_count']] = df['dialogue'].apply(
    lambda x: pd.Series(calculate_counts(x))
)

stats = df.groupby(["season", "episode"]).agg(
    total_sentences =("sentence_count", "sum"),
    total_words = ("word_count", "sum"),
).reset_index()

stats["avg_sentences_per_episode"] = stats["sentence_count"]
stats["avg_words_per_episode"] = stats["word_count"]

season_stats = stats.groupby(["season"]).agg(
    total_sentences =("avg_sentences_per_episode", "mean"),
    total_words = ("avg_words_per_episode", "mean"),
).reset_index()


KeyError: 'sentence_count'

In [ ]:
stats

In [33]:
season_stats

,season,total_sentences,total_words
0,03,2.000000,25.200000
1,04,1.725212,16.900850
2,05,1.740385,17.807692
3,06,1.628842,15.163121
4,07,1.468504,13.488189
5,08,1.492473,14.470968
6,09,1.424908,12.783883
7,10,1.366477,12.720170


In [ ]:
# tokenize again for counting
def calculate_counts(dialogue):
    sentences = nltk.sent_tokenize(dialogue)
    words = nltk.word_tokenize(dialogue)
    return len(sentences), len(words)

# Apply to function
df[['sentence_count', 'word_count']] = df['dialogue'].apply(
    lambda x: pd.Series(calculate_counts(x))
)

# Extract season and episode information
df['season'] = df['episode_name'].str.extract(r'Series (\d+)')
df['episode'] = df['episode_name'].str.extract(r'Episode (\d+)')

# Group by season and episode
character_episode_stats = df.groupby(['season', 'episode']).agg(
    total_sentences=('sentence_count', 'sum'),
    total_words=('word_count', 'sum'),
).reset_index()

# Calculate per season avg
season_stats = character_episode_stats.groupby(['season']).agg(
    avg_sentences_per_episode=('total_sentences', 'mean'),
    avg_words_per_episode=('total_words', 'mean')
).reset_index()

# Calculate per season deviations
season_dev = character_episode_stats.groupby(['season']).agg(
    sentences_per_episode_dev=('total_sentences', 'mean'),
    words_per_episode_dev=('total_words', 'mean')
).reset_index()

In [50]:
print(season_stats)

  season  avg_sentences_per_episode  avg_words_per_episode
0     03                  10.000000             126.000000
1     04                  40.600000             397.733333
2     05                  34.476190             352.761905
3     06                  31.318182             291.545455
4     07                  31.083333             285.500000
5     08                  28.916667             280.375000
6     09                  33.826087             303.478261
7     10                  40.083333             373.125000
